In [17]:
with open("inputs/Day_19_1.txt") as f:
    puzzle_data = f.read()
    
def part_1_solution(raw_data):
    raw_rules, messages = raw_data.split('\n\n')
    parsed_rules, evaluated_rules = parse_rules(raw_rules)
    allowed_messages = set(evaluate_rule('0', parsed_rules, evaluated_rules))

    cnt = 0
    for message in messages.splitlines():
        if message in allowed_messages:
            cnt += 1
    
    return cnt


def parse_rules(raw_rules):
    parsed_rules = dict()
    evaluated_rules = dict()
    
    for raw_rule in raw_rules.splitlines():
        rule_idx, child_rules = raw_rule.split(":")
        child_rules = child_rules.strip()
        # check if rule is already evaluated
        if child_rules.startswith('"'):
            evaluated_rule = child_rules[1:-1]
            evaluated_rules[rule_idx] = [evaluated_rule]
        else:
            parsed_rules[rule_idx] = child_rules
    
    return parsed_rules, evaluated_rules


def evaluate_rule(rule_idx, parsed_rules, evaluated_rules):
    if rule_idx in evaluated_rules:
        return evaluated_rules[rule_idx]
    
    raw_rules_idx = parsed_rules[rule_idx].split('|')
    rules = list()
    
    for current_rules_child_idx in raw_rules_idx:
        current_rules_child_idx = current_rules_child_idx.strip().split(" ")
        
        current_rules = evaluate_rule(current_rules_child_idx[0], parsed_rules, evaluated_rules)
        idx = 1
        while idx < len(current_rules_child_idx):
            child_rules = evaluate_rule(current_rules_child_idx[idx], parsed_rules, evaluated_rules)
            current_rules = concatinate_rules(current_rules, child_rules)
            idx += 1
        
        rules.extend(current_rules)
        
    evaluated_rules[rule_idx] = rules
    
    return rules

            
def concatinate_rules(base_rules, child_rules):
    concatinated_rules = list()
    for base_rule in base_rules:
        for child_rule in child_rules:
            result_rule = base_rule + child_rule
            concatinated_rules.append(result_rule)
            
    return concatinated_rules

In [18]:
from helpers import test_single_case

test_input = """\
0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b"

ababbb
bababa
abbbab
aaabbb
aaaabbb\
"""

test_single_case(part_1_solution, 2, test_input)

PASSED (in 0.02 [ms])


In [19]:
%%time
print(f"Part 1 solution: {part_1_solution(puzzle_data)}")

Part 1 solution: 222
CPU times: user 592 ms, sys: 39.8 ms, total: 632 ms
Wall time: 628 ms


In [25]:
from pprint import pprint

with open("inputs/Day_19_2.txt") as f:
    puzzle_data = f.read()
    
def part_2_solution(raw_data):
    raw_rules, messages = raw_data.split('\n\n')
    parsed_rules, evaluated_rules = parse_rules(raw_rules)
#     rules_for_42 = evaluate_rule('0', parsed_rules, evaluated_rules)
#     return len(max(rules_for_42, key=lambda x: len(x)))
#     return len(set(rules_for_42))
    rules_for_0 = evaluate_rule('0', parsed_rules, evaluated_rules)
    pprint(rules_for_0)
    print('-' * 10)
#     print('All rules evaluated')
    allowed_messages = set(rules_for_0)
    print(len(allowed_messages))

    cnt = 0
    for message in messages.splitlines():
        print(message, end='    ')
        if message in allowed_messages:
            print("MATCH")
            cnt += 1
        else:
            print("NOT MATCH")
    
    return cnt


def parse_rules(raw_rules):
    parsed_rules = dict()
    evaluated_rules = dict()
    
    for raw_rule in raw_rules.splitlines():
        rule_idx, child_rules = raw_rule.split(":")
        child_rules = child_rules.strip()
        # check if rule is already evaluated
        if child_rules.startswith('"'):
            evaluated_rule = child_rules[1:-1]
            evaluated_rules[rule_idx] = [evaluated_rule]
        else:
            parsed_rules[rule_idx] = child_rules
    
    return parsed_rules, evaluated_rules


def evaluate_rule(rule_idx, parsed_rules, evaluated_rules):
    if rule_idx in evaluated_rules:
        return evaluated_rules[rule_idx]
    
#     print(f"Evaluating {rule_idx} ...")
    
    raw_rules_idx = parsed_rules[rule_idx].split('|')
    rules = list()
#     evaluated_rules[rule_idx] = rules
    
    for current_rules_child_idx in raw_rules_idx:
        current_rules_child_idx = current_rules_child_idx.strip().split(" ")
        
        current_rules = evaluate_rule(current_rules_child_idx[0], parsed_rules, evaluated_rules)
#         rules.extend(current_rules)
        idx = 1
        while idx < len(current_rules_child_idx):
            child_rules = evaluate_rule(current_rules_child_idx[idx], parsed_rules, evaluated_rules)
            current_rules = concatinate_rules(current_rules, child_rules)
            idx += 1
        
        rules.extend(current_rules)
        evaluated_rules[rule_idx] = rules
        
    evaluated_rules[rule_idx] = rules
    
#     print(f"{rule_idx} evaluated")
    
    return rules

            
def concatinate_rules(base_rules, child_rules):
    concatinated_rules = list()
    for base_rule in base_rules:
        for child_rule in child_rules:
            result_rule = base_rule + child_rule
            concatinated_rules.append(result_rule)
            
    return concatinated_rules

In [4]:
import re


with open("inputs/Day_19_2.txt") as f:
    puzzle_data = f.read()
    

def part_2_solution(s: str) -> int:
    rules, lines = s.split('\n\n')

    rules_s = {}
    for line in rules.splitlines():
        k, _, v = line.partition(': ')
        rules_s[k] = v

    def _get_re(s: str) -> str:
        if s == '|':
            return s

        rule_s = rules_s[s]
        if rule_s.startswith('"'):
            return rule_s.strip('"')
        else:
            return f'({"".join(_get_re(part) for part in rule_s.split())})'

    re_42 = re.compile(_get_re('42'))
    re_31 = re.compile(_get_re('31'))

    count = 0
    for line in lines.splitlines():
        pos = 0

        count_42 = 0
        while match := re_42.match(line, pos):
            count_42 += 1
            pos = match.end()

        count_31 = 0
        while match := re_31.match(line, pos):
            count_31 += 1
            pos = match.end()

        if 0 < count_31 < count_42 and pos == len(line):
            count += 1

    return count

In [5]:
from helpers import test_multiple_cases

test_input_1 = """\
42: 9 14 | 10 1
9: 14 27 | 1 26
10: 23 14 | 28 1
1: "a"
11: 42 31
5: 1 14 | 15 1
19: 14 1 | 14 14
12: 24 14 | 19 1
16: 15 1 | 14 14
31: 14 17 | 1 13
6: 14 14 | 1 14
2: 1 24 | 14 4
0: 8 11
13: 14 3 | 1 12
15: 1 | 14
17: 14 2 | 1 7
23: 25 1 | 22 14
28: 16 1
4: 1 1
20: 14 14 | 1 15
3: 5 14 | 16 1
27: 1 6 | 14 18
14: "b"
21: 14 1 | 1 14
25: 1 1 | 1 14
22: 14 14
8: 42
26: 14 22 | 1 20
18: 15 15
7: 14 5 | 1 21
24: 14 1

abbbbbabbbaaaababbaabbbbabababbbabbbbbbabaaaa
bbabbbbaabaabba
babbbbaabbbbbabbbbbbaabaaabaaa
aaabbbbbbaaaabaababaabababbabaaabbababababaaa
bbbbbbbaaaabbbbaaabbabaaa
bbbababbbbaaaaaaaabbababaaababaabab
ababaaaaaabaaab
ababaaaaabbbaba
baabbaaaabbaaaababbaababb
abbbbabbbbaaaababbbbbbaaaababb
aaaaabbaabaaaaababaa
aaaabbaaaabbaaa
aaaabbaabbaaaaaaabbbabbbaaabbaabaaa
babaaabbbaaabaababbaabababaaab
aabbbbbaabbbaaaaaabbbbbababaaaaabbaaabba\
"""

test_input_2 = """\
42: 9 14 | 10 1
9: 14 27 | 1 26
10: 23 14 | 28 1
1: "a"
11: 42 31 | 42 11 31
5: 1 14 | 15 1
19: 14 1 | 14 14
12: 24 14 | 19 1
16: 15 1 | 14 14
31: 14 17 | 1 13
6: 14 14 | 1 14
2: 1 24 | 14 4
0: 8 11
13: 14 3 | 1 12
15: 1 | 14
17: 14 2 | 1 7
23: 25 1 | 22 14
28: 16 1
4: 1 1
20: 14 14 | 1 15
3: 5 14 | 16 1
27: 1 6 | 14 18
14: "b"
21: 14 1 | 1 14
25: 1 1 | 1 14
22: 14 14
8: 42 | 42 8
26: 14 22 | 1 20
18: 15 15
7: 14 5 | 1 21
24: 14 1

abbbbbabbbaaaababbaabbbbabababbbabbbbbbabaaaa
bbabbbbaabaabba
babbbbaabbbbbabbbbbbaabaaabaaa
aaabbbbbbaaaabaababaabababbabaaabbababababaaa
bbbbbbbaaaabbbbaaabbabaaa
bbbababbbbaaaaaaaabbababaaababaabab
ababaaaaaabaaab
ababaaaaabbbaba
baabbaaaabbaaaababbaababb
abbbbabbbbaaaababbbbbbaaaababb
aaaaabbaabaaaaababaa
aaaabbaaaabbaaa
aaaabbaabbaaaaaaabbbabbbaaabbaabaaa
babaaabbbaaabaababbaabababaaab
aabbbbbaabbbaaaaaabbbbbababaaaaabbaaabba\
"""

test_multiple_cases(
    part_2_solution,
    (
        (3, test_input_1),
        (12, test_input_2)
    )
)

Case #0: FAILED, expected 3, got 12 (0.18 [ms])
Case #1: PASSED (0.19 [ms])
Passed: 1; Failed: 1; Elapsed time: 1.25 [ms]


In [6]:
%%time
print(f"Part 2 solution: {part_2_solution(puzzle_data)}")

Part 2 solution: 339
CPU times: user 16.2 ms, sys: 0 ns, total: 16.2 ms
Wall time: 14.9 ms
